In [4]:
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.ML.DataView"

Installed Packages Microsoft.ML, 4.0.2 Microsoft.ML.DataView, 4.0.2

## 1) Import thư viện cần thiết

In [5]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.Text;
using Microsoft.ML.Trainers;


## 2) Định nghĩa class cho dữ liệu đầu vào và kết quả

Giải thích: Định nghĩa `ReviewInput` chứa các thuộc tính mapped tới cột file TSV, và `ReviewPrediction` chứa output từ mô hình.
Đặt các lớp này trong một ô code riêng để kernel load chúng dễ dàng.

In [6]:
public class ReviewInput
{
    [LoadColumn(0)]
    public string Text { get; set; }

    [LoadColumn(1), ColumnName("Label")]
    public bool Label { get; set; }
}

public class ReviewPrediction
{
    [ColumnName("PredictedLabel")]
    public bool Prediction { get; set; }

    public float Probability { get; set; }
    public float Score { get; set; }
}


## 3) Khởi tạo MLContext

Giải thích: `MLContext` là entry point — giữ trạng thái cho các thao tác ML.NET (random seed, logging...). Tách ô này nếu bạn muốn thay seed hoặc cài đặt khác.

In [7]:
var mlContext = new MLContext(seed: 42);
Console.WriteLine("MLContext created with seed=42");

MLContext created with seed=42


## 4) Tạo / Load dữ liệu mẫu

Giải thích: Ô code này sẽ đảm bảo thư mục `Data/` có hai file `train.tsv` và `test.tsv`. Nếu bạn đã có dữ liệu thực, chỉ cần chỉnh đường dẫn và bỏ phần tạo file.

In [ ]:
var trainPath = Path.Combine("Data", "train.tsv");
var testPath = Path.Combine("Data", "test.tsv");

var trainDataView = mlContext.Data.LoadFromTextFile<ReviewInput>(trainPath, hasHeader: true, separatorChar: '\t');
var testDataView = mlContext.Data.LoadFromTextFile<ReviewInput>(testPath, hasHeader: true, separatorChar: '\t');

//using trainTestSplit 
var dataPath = Path.Combine("Data", "train.tsv");
var data = mlContext.Data.LoadFromTextFile<ReviewInput>(dataPath, hasHeader: true, separatorChar: '\t');
var split = mlContext.Data.TrainTestSplit(data, testFraction: 0.2, samplingKeyColumnName: nameof(ReviewInput.Label), seed: 123);
var trainSet = split.TrainSet;
var testSet  = split.TestSet;

Console.WriteLine("Data loaded into IDataView objects.");

Data loaded into IDataView objects.


## 5) Xây dựng Pipeline

Giải thích: Tách rõ phần transform và trainer. Ở đây ta dùng `FeaturizeText` → `NormalizeMinMax` → `Concatenate` → `SdcaLogisticRegression`.
Bạn có thể thử thay đổi normalizer hoặc trainer ở ô này.

In [75]:
var pipeline = mlContext.Transforms.Text
    // FeaturizeText: chuyển chuỗi text thành vector số (n-grams, TF-IDF, bag-of-words...). 
    // -> Tạo cột đầu ra "TextFeaturized" chứa các đặc trưng số cho văn bản.
    .FeaturizeText(outputColumnName: "TextFeaturized", inputColumnName: nameof(ReviewInput.Text))

    // NormalizeMinMax: chuẩn hóa các giá trị trong "TextFeaturized" về khoảng [0,1].
    // -> Giúp các thuật toán (như logistic regression, SVM, ...) hội tụ ổn định hơn,
    //    và tránh rằng các chiều có magnitude lớn áp đảo các chiều khác.
    .Append(mlContext.Transforms.NormalizeMinMax("TextFeaturized"))

    // Concatenate: gộp một hoặc nhiều cột feature vào cột duy nhất "Features". (định dạng mà hầu hết trainer yêu cầu)
    // -> Các trainers của ML.NET mặc định nhận input là cột "Features" (vector duy nhất).
    //    Ở đây chúng ta chỉ gộp "TextFeaturized" (có thể thêm cột numeric khác nếu có).
    .Append(mlContext.Transforms.Concatenate("Features", "TextFeaturized"))

    // Trainer: sdca logistic regression cho binary classification.
    // - labelColumnName: tên cột chứa nhãn (ở đây là "Label").
    // - featureColumnName: tên cột features (ở đây dùng "Features" vừa tạo).
    // -> Kết quả là một estimator/transformer huấn luyện mô hình phân loại nhị phân.
    .Append(mlContext.BinaryClassification.Trainers.SdcaLogisticRegression(
        labelColumnName: "Label", featureColumnName: "Features"));

Console.WriteLine("Pipeline created.");

Pipeline created.


## 6) Huấn luyện mô hình

Giải thích: Fit pipeline trên dữ liệu training. Kết quả là `ITransformer` (model) gồm cả transforms và trainer.

In [76]:
// Fit là lệnh kích hoạt huấn luyện/“fit” của một Estimator (ở đây là pipeline) trên dữ liệu (trainDataView).
// Fit biến chuỗi các bước mô tả (Estimator pipeline) thành một pipeline thực thi (Transformer) dùng để dự đoán.
var model = pipeline.Fit(trainDataView);
Console.WriteLine("Model training completed.");

Model training completed.


## 7) Đánh giá mô hình trên test set

Giải thích: Transform test data và dùng `Evaluate` để lấy metrics Accuracy, AUC... Ô code ở dưới sẽ in ra các chỉ số chính.

In [77]:
// Dùng model đã huấn luyện để chuyển testDataView thành tập dự đoán (scored data)
var predictions = model.Transform(testDataView);

// Đánh giá bộ dự đoán bằng hàm Evaluate của MLContext.
// - predictions: IDataView sau khi đã có các cột do model sinh (Score, Probability, PredictedLabel...).
// - labelColumnName: tên cột chứa nhãn thực tế trong test set (ở ví dụ này là "Label").
var metrics = mlContext.BinaryClassification.Evaluate(predictions, labelColumnName: "Label");
Console.WriteLine($"Accuracy: {metrics.Accuracy:P2}");
Console.WriteLine($"AUC: {metrics.AreaUnderRocCurve:P2}");

Accuracy: 100.00%
AUC: 100.00%


## 8) Tạo Prediction Engine (single prediction)

Giải thích: `PredictionEngine<TSrc,TDest>` phù hợp cho trường hợp dự đoán từng bản ghi (không dùng cho high-concurrency trên server).

In [78]:
var engine = mlContext.Model.CreatePredictionEngine<ReviewInput, ReviewPrediction>(model);
Console.WriteLine("Prediction engine created.");

Prediction engine created.


## 9) Dự đoán mẫu

Giải thích: Dùng `engine.Predict` trên một sample để kiểm tra kết quả dự đoán và score.

In [79]:
var sample = new ReviewInput { Text = "Quá tuyệt vời, dịch vụ rất chuyên nghiệp" };
var pred = engine.Predict(sample);
Console.WriteLine($"Prediction: {pred.Prediction}, Probability: {pred.Probability:P2}, Score: {pred.Score}");

Prediction: True, Probability: 99.12%, Score: 4.728811


## 10) Lưu mô hình


In [ ]:
var modelPath = Path.Combine("ExportedModels", "reviewModel.zip");
mlContext.Model.Save(model, trainDataView.Schema, modelPath);
Console.WriteLine($"Model saved to {modelPath}");

Model saved to MLModels\reviewModel.zip
